In [2]:
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import warnings
from scipy.fftpack import fft, ifft
warnings.filterwarnings("ignore")

In [ ]:
import optiver2023
#env = optiver2023.make_env()
#iter_test = env.iter_test()

counter = 0
y_min, y_max = -64, 64
qps, predictions = [], []
cache = pd.DataFrame()

for (test, revealed_targets, sample_prediction) in iter_test:
    a = test
a

In [ ]:
stop

In [ ]:
from itertools import combinations
def feat_eng_RANGE(df):
    def get_RANGE(data,i):
        positive_sum = data[i].diff().clip(lower=0).rolling(window=10, min_periods=10).sum()
        absolute_sum = data[i].diff().abs().rolling(window=10, min_periods=10).sum()
        factor = positive_sum / absolute_sum
        factor = factor.fillna(0.5) 
        factor.dropna(inplace=True)
        return factor
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    
    prices = ['bid_price','ask_price','wap','reference_price']#range_reference_price 0.071838 range_ask_price 0.133756 range_bid_price 0.123905 range_wap 0.098298
    for i in prices:
        fac=df[['stock_id','date_id',i]].groupby(['stock_id','date_id']).apply(lambda x:get_RANGE(x,i))
        fac=fac.reset_index().set_index('level_2').sort_index()
        df['range_'+i]=np.nan
        df['range_'+i].iloc[fac.index]=fac[0]
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
    
    return df.replace([np.inf, -np.inf], 0)

In [ ]:
from itertools import combinations
def feat_eng_voi(df):
    def voi(df):
        bid_sub_price = df['bid_price'] - df['bid_price'].shift(1)
        ask_sub_price = df['ask_price'] - df['ask_price'].shift(1)
        bid_sub_volume = df['bid_size'] - df['bid_size'].shift(1)
        ask_sub_volume = df['ask_size'] - df['ask_size'].shift(1)
        bid_volume_change = bid_sub_volume
        ask_volume_change = ask_sub_volume
        # bid_volume_change[bid_sub_price == 0] = bid_sub_volume[bid_sub_price == 0]
        bid_volume_change[bid_sub_price < 0] = 0
        bid_volume_change[bid_sub_price > 0] = df['bid_size'][bid_sub_price > 0]
        ask_volume_change[ask_sub_price < 0] = df['ask_size'][ask_sub_price < 0]
        ask_volume_change[ask_sub_price > 0] = 0
        tick_fac_data = (bid_volume_change - ask_volume_change)/(df['bid_size']+ df['bid_size']).rolling(5).sum()
        return tick_fac_data
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    
    fac=df.groupby(['stock_id','date_id']).apply(voi)
    fac=fac.reset_index().set_index('level_2').sort_index()
    df['voi']=np.nan
    df['voi'].iloc[fac.index]=fac[0]

    df.drop(columns=[
        'date_id', 
    ], inplace=True)
    
    return df.replace([np.inf, -np.inf], 0)

In [ ]:
from itertools import combinations
def feat_eng_sin(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for c in combinations(sizes, 2):
        df[f'{c[0]}_logminus_{c[1]}'] = (np.log(df[f'{c[0]}']) - np.log(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_sinminus_{c[1]}'] = (df[f'{c[0]}'] - np.sin(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_tanminus_{c[1]}'] = (df[f'{c[0]}'] - np.tan(df[f'{c[1]}'])).astype(np.float32)
    
    
    #所有价格之间做差或求和
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_logminus_{c[1]}'] = (np.log(df[f'{c[0]}']) - np.log(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_sinminus_{c[1]}'] = (df[f'{c[0]}'] - np.sin(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_tanminus_{c[1]}'] = (df[f'{c[0]}'] - np.tan(df[f'{c[1]}'])).astype(np.float32)
    
    #计算统计指标
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    #计算所有价格和交易量的滞后项和pct
    for col in sizes+prices:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
            
    df.drop(columns=[
        'date_id', 
    ], inplace=True)

In [ ]:
import pandas as pd

In [ ]:
"""
train为初始训练集
"""
train=pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
max_sizes = train.groupby('stock_id')['bid_size'].max() + train.groupby('stock_id')['ask_size'].max()
min_sizes = train.groupby('stock_id')['bid_size'].min() + train.groupby('stock_id')['ask_size'].min()
mean_sizes = train.groupby('stock_id')['bid_size'].mean() + train.groupby('stock_id')['ask_size'].mean()
first_sizes = train.groupby('stock_id')['bid_size'].first() + train.groupby('stock_id')['ask_size'].first()
last_sizes = train.groupby('stock_id')['bid_size'].last() + train.groupby('stock_id')['ask_size'].last()
from itertools import combinations
def feat_eng_unbalanced(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    #size类指标
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df["volume"] = df.eval("ask_size + bid_size")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())    
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())
    
    df['high_volume'] = np.where(df['volume'] > df['median_size'], 1, 0)
    
    
    df['ask_x_size'] = (df.eval('ask_size*ask_price')-df.eval('bid_size*bid_price'))/df.eval('ask_size*ask_price')+df.eval('bid_size*bid_price')
    #对所有size和price计算两方不平衡和三方不平衡以及做差
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for c in combinations(sizes, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(sizes, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
    
    #所有价格之间做差或求和
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
    
    return df.replace([np.inf, -np.inf], 0)

In [ ]:
%%time
data_train=feat_eng_unbalanced(train)

data_train

In [ ]:
data_train

In [ ]:
train

In [ ]:
feat_eng_unbalanced(pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv"))

In [ ]:
pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv")

In [ ]:
import optiver2023
#env = optiver2023.make_env()
#iter_test = env.iter_test()
a = pd.DataFrame()
counter = 0
y_min, y_max = -64, 64
qps, predictions = [], []
cache = pd.DataFrame()

for (test, revealed_targets, sample_prediction) in iter_test:
    a = test
a

In [ ]:
import optiver2023
#env = optiver2023.make_env()
#iter_test = env.iter_test()

counter = 0
y_min, y_max = -64, 64
qps, predictions = [], []
cache = pd.DataFrame()

for (test, revealed_targets, sample_prediction) in iter_test:
    a = test
a

In [4]:
train = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [8]:
from itertools import combinations
def feat_engineering(df):
    def voi(df):
        bid_sub_price = df['bid_price'] - df['bid_price'].shift(1)
        ask_sub_price = df['ask_price'] - df['ask_price'].shift(1)
        bid_sub_volume = df['bid_size'] - df['bid_size'].shift(1)
        ask_sub_volume = df['ask_size'] - df['ask_size'].shift(1)
        bid_volume_change = bid_sub_volume
        ask_volume_change = ask_sub_volume
        # bid_volume_change[bid_sub_price == 0] = bid_sub_volume[bid_sub_price == 0]
        bid_volume_change[bid_sub_price < 0] = 0
        bid_volume_change[bid_sub_price > 0] = df['bid_size'][bid_sub_price > 0]
        ask_volume_change[ask_sub_price < 0] = df['ask_size'][ask_sub_price < 0]
        ask_volume_change[ask_sub_price > 0] = 0
        tick_fac_data = (bid_volume_change - ask_volume_change)/(df['bid_size']+ df['bid_size']).rolling(5).sum()
        return tick_fac_data
    
    def get_RANGE(data,i):
        factor = pd.DataFrame(index=data.index)
        positive_sum = data[i].diff().clip(lower=0).rolling(window=10, min_periods=10).sum()
        absolute_sum = data[i].diff().abs().rolling(window=10, min_periods=10).sum()
        factor['RANGE'] = positive_sum / absolute_sum
        factor['RANGE'] = factor['RANGE'].fillna(0.5) 
        factor.dropna(inplace=True)
        return factor['RANGE']
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    prices = ['bid_price','ask_price','wap','reference_price']#range_reference_price 0.071838 range_ask_price 0.133756 range_bid_price 0.123905 range_wap 0.098298
    for i in prices:
        fac=df[['stock_id','date_id',i]].groupby(['stock_id','date_id']).apply(lambda x:get_RANGE(x,i))
        fac=fac.reset_index().set_index('level_2').sort_index()
        df['range_'+i]=np.nan
        df['range_'+i].iloc[fac.index]=fac['RANGE']
    print("RANGE FINISH")
    fac=df.groupby(['stock_id','date_id']).apply(voi)
    fac=fac.reset_index().set_index('level_2').sort_index()
    df['voi']=np.nan
    df['voi'].iloc[fac.index]=fac[0]
    print("VOI FINIAH")
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    for c in combinations(sizes, 2):
        df[f'{c[0]}_logminus_{c[1]}'] = (np.log(df[f'{c[0]}']) - np.log(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_sinminus_{c[1]}'] = (df[f'{c[0]}'] - np.sin(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_tanminus_{c[1]}'] = (df[f'{c[0]}'] - np.tan(df[f'{c[1]}'])).astype(np.float32)
    #所有价格之间做差或求和
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    for c in combinations(prices, 2):
        df[f'{c[0]}_logminus_{c[1]}'] = (np.log(df[f'{c[0]}']) - np.log(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_sinminus_{c[1]}'] = (df[f'{c[0]}'] - np.sin(df[f'{c[1]}'])).astype(np.float32)
        df[f'{c[0]}_tanminus_{c[1]}'] = (df[f'{c[0]}'] - np.tan(df[f'{c[1]}'])).astype(np.float32)
    #计算统计指标
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    #计算所有价格和交易量的滞后项和pct
    for col in sizes+prices:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    print("SIN FINISH")
    
    median_sizes = df.groupby('stock_id')['bid_size'].median() + df.groupby('stock_id')['ask_size'].median()
    std_sizes = df.groupby('stock_id')['bid_size'].std() + df.groupby('stock_id')['ask_size'].std()
    max_sizes = df.groupby('stock_id')['bid_size'].max() + df.groupby('stock_id')['ask_size'].max()
    min_sizes = df.groupby('stock_id')['bid_size'].min() + df.groupby('stock_id')['ask_size'].min()
    mean_sizes = df.groupby('stock_id')['bid_size'].mean() + df.groupby('stock_id')['ask_size'].mean()
    first_sizes = df.groupby('stock_id')['bid_size'].first() + df.groupby('stock_id')['ask_size'].first()
    last_sizes = df.groupby('stock_id')['bid_size'].last() + df.groupby('stock_id')['ask_size'].last()
    
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df["volume"] = df.eval("ask_size + bid_size")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())    
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())
    
    df['high_volume'] = np.where(df['volume'] > df['median_size'], 1, 0)
    
    
    df['ask_x_size'] = (df.eval('ask_size*ask_price')-df.eval('bid_size*bid_price'))/df.eval('ask_size*ask_price')+df.eval('bid_size*bid_price')
    #对所有size和price计算两方不平衡和三方不平衡以及做差
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for c in combinations(sizes, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(sizes, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
    
    #所有价格之间做差或求和
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
    
    return df.replace([np.inf, -np.inf], 0)


In [ ]:
feat_all = feat_engineering(train)
feat_all

RANGE FINISH
VOI FINIAH
SIN FINISH
